# QuickSight API 

We are going to test the QuickSight API working with a classic dataset in SKlearn (a Python library), store it in S3, then import it as a dataset in QuickSighty to be able to generate our dashboards. To do this we use the Python SDK development kit, Boto3.


1. [Prequisites](#Prequisites)
    * [Import Library](#Library)
    * [global variables](#globalvariables)
    * [Dataset](#Dataset)
    * [Upload dataset to S3](#Upload_dataset)
    * [Upload manifest to S3](#Upload_manifest)
2. [Quicksight](#Quicksight)
    * [Generate DataSource](#DataSource)
    * [Generate DataSet](#DataDataSet)
    * [Generate Analysis](#DataAnalysis)
    * [Generate Template](#DataTemplate)
    * [Generate Dashboard](#DataDashboard)
    * [Drop Resources](#Dropresources)
        
    
        

<a id='Prequisites'></a>

## Prequisites
---

<a id='Library'></a>
+ ### Import Library

In [ ]:
import boto3
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import json

<a id='globalvariables'></a>
+ ### Global variables

We define global variables that we will use throughout the notebook, we also take the AWS user account credentials and the quickSight username from the json_credentials.json file that we will need to connect to QuickSight.


In [ ]:
# S3
bucket='quicksight-api'
region_name = 'us-east-1'
name_data = 'iris.csv'
uri_data_s3 = 's3://{}/data/{}'.format(bucket,name_data)
key_data_s3 = 'data/{}'.format(name_data)
name_manifest = 'manifest_iris'
uri_manifest_s3 = 's3://{}/manifest/{}.json'.format(bucket,name_manifest)
key_manifest_s3 = 'manifest/{}.json'.format(name_manifest)

# quicksight
DataSourceId = 'DataSourceID-SDK'
name_DataSource = 'DataSource-SDK'

DataSetId = 'data_irisID_sdk'
name_DataSet = 'data_iris_sdk'

dataset_template_name = 'iris (2)'
analysis_template_name = 'analysis_template_iris'
TemplateId ='template_irisID_sdk'
name_Template ='template_iris_sdk'
Versiontemplate = '1'
    
AnalysisId = 'analysis_iris_ID_sdk'
name_Analysis = 'analysis_iris_sdk'

DashboardId = 'dashboard_iris_ID_sdk'
name_Dashboard = 'dashboard_iris_sdk'
VersionDashboard = '1'

# credentials
with open('json_credentials.json', 'r') as file:
    credentials = json.load(file)
    
AwsAccountId = credentials['AwsAccountId']
name_user = credentials['name_user']
arn_user = f"arn:aws:quicksight:{region_name}:{AwsAccountId}:user/default/{name_user}"

<a id='Dataset'></a>
+ ### Dataset

In this case we are going to use a well-known dataset included in the SKlearn library in order to show how we can generate programmatically a dashboard in Quicksight after storing it in S3.


In [ ]:
# read dataset iris to sklearn
iris = load_iris()
data = pd.concat([pd.DataFrame(iris.data),pd.DataFrame(iris.target)], axis=1)
data.columns = iris.feature_names + ['labels']
data.labels = data.labels.astype('O')

In [ ]:
data.head()

In [ ]:
data.info()

<a id='Upload_dataset'></a>
+ ### Upload dataset to S3

We create a bucket in S3 and upload our dataset to S3 from the variables **bucket**, **name_data** and **uri_data_s3**.


In [ ]:
# create session s3
client = boto3.client('s3')

# create bucket
client.create_bucket(
    Bucket=bucket,
)

In [ ]:
data.to_csv('iris.csv',index=False)

# update iris to s3
client.upload_file('iris.csv', bucket, key_data_s3)

<a id='Upload_manifest'></a>
+ ### Upload manifest to S3

We create a manifest file with information related to the location in S3 and the characteristics of the dataset. Then we upload it to S3.


In [ ]:
# create manifest
import json

json_object = {}
json_object['fileLocations'] = []

json_object['fileLocations'].append({'URIs':['{}'.format(uri_data_s3)]})
json_object['globalUploadSettings'] = {}
json_object['globalUploadSettings']['format'] = 'CSV'
json_object['globalUploadSettings']['delimiter'] = ','
json_object['globalUploadSettings']['textqualifier'] = "'"
json_object['globalUploadSettings']['containsHeader'] = 'true'            

with open('json_object.json', 'w') as file:
    json.dump(json_object, file, indent=4)

In [ ]:
# update manifest to s3
client.put_object(
     Body=json.dumps(json_object),
     Bucket=bucket,
     Key=key_manifest_s3
)

<a id='Quicksight'></a>

## QuickSight
---

<a id='DataSource'></a>
+ ### Generate DataSource

Firstly we generate the connection with QuickSight and then with our datasource, which is S3 in this case.

In this case we use the function [create_data_source](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.create_data_source). After creation, we list the DataSource and verify that it was created correctly using the function [list_data_sources](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.list_data_sources), It can also be done by command line by the following syntax.

`!aws quicksight list-data-sources --region $region_name --aws-account-id $AwsAccountId`

In [ ]:
# create session quicksight
qs_client = boto3.client('quicksight',
        region_name=region_name)

In [ ]:
# create data source 
Response_datasource = qs_client.create_data_source(
    AwsAccountId=AwsAccountId, 
    DataSourceId=DataSourceId,
    Name=name_DataSource,
    Type='S3',
    DataSourceParameters={
        'S3Parameters':{
            'ManifestFileLocation':{
                'Bucket':bucket,
                'Key':key_manifest_s3
            }
        }
    },
   Permissions=[
        {
            "Principal":f'{arn_user}',
            "Actions": [
                "quicksight:DescribeDataSource",
                "quicksight:DescribeDataSourcePermissions",
                "quicksight:PassDataSource",
                "quicksight:UpdateDataSource",
                "quicksight:UpdateDataSourcePermissions",
                "quicksight:DeleteDataSource"
            ]
        }
    ]
)
Response_datasource

In [ ]:
# We verify that the datasource was created
Response_list_datasource = qs_client.list_data_sources(AwsAccountId=AwsAccountId )
np.sum([np.array([Response_list_datasource['DataSources'][i]['Name'] for i in np.arange(len(Response_list_datasource['DataSources']))])==name_DataSource])==1

<a id='DataDataSet'></a>
+ ### Generate DataSet

In this case we use the function [create_data_set](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.create_data_set). After creation, we list the DataSet of the account and verify that it was created correctly using the function [list_data_set](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.list_data_sources), it can also be done on the command line by the following syntax.

`!aws quicksight list-data-sets --region $region_name --aws-account-id $AwsAccountId`

In [ ]:
Response_dataset = qs_client.create_data_set(
    AwsAccountId=AwsAccountId,
    DataSetId=DataSetId,
    Name=name_DataSet,
    PhysicalTableMap={
        'iris':{
            'S3Source':{
                'DataSourceArn':Response_datasource['Arn'],
                "InputColumns": [
                    {
                        'Name':'sepal length (cm)',
                        'Type':"STRING"
                    },
                    {
                        'Name':'sepal width (cm)',
                        'Type':"STRING"
                    },
                    {
                        'Name':'petal length (cm)',
                        'Type':"STRING"
                    },
                    {
                        'Name':'petal width (cm)',
                        'Type':"STRING"
                    },
                    {
                        'Name':'labels',
                        'Type':"STRING"
                    }
                ]
            }
        }
    },
    LogicalTableMap={
        'iris':{
            'Alias':'iris_alias',
            'DataTransforms':[
                {
                    'CastColumnTypeOperation':{
                        'ColumnName':'sepal length (cm)',
                        'NewColumnType':'DECIMAL'
                    }
                },
                {
                    'CastColumnTypeOperation':{
                        'ColumnName':'sepal width (cm)',
                        'NewColumnType':'DECIMAL'
                    }
                },
                {
                    'CastColumnTypeOperation':{
                        'ColumnName':'petal length (cm)',
                        'NewColumnType':'DECIMAL'
                    }
                },
                {
                    'CastColumnTypeOperation':{
                        'ColumnName':'petal width (cm)',
                        'NewColumnType':'DECIMAL'
                    }
                },
            ],
            'Source':{
                'PhysicalTableId':'iris'
            }
        }
    },
    ImportMode="SPICE",
    Permissions=[
        {
            "Principal":arn_user,
            "Actions": [
                "quicksight:UpdateDataSetPermissions",
                "quicksight:DescribeDataSet",
                "quicksight:DescribeDataSetPermissions",
                "quicksight:PassDataSet",
                "quicksight:DescribeIngestion",
                "quicksight:ListIngestions",
                "quicksight:UpdateDataSet",
                "quicksight:DeleteDataSet",
                "quicksight:CreateIngestion",
                "quicksight:CancelIngestion"
            ]
        }
    ]  
)

Response_dataset

In [ ]:
# We verify that the dataset was created
Response_list_dataset = qs_client.list_data_sets(AwsAccountId=AwsAccountId)
np.sum([np.array([Response_list_dataset['DataSetSummaries'][i]['Name'] for i in np.arange(len(Response_list_dataset['DataSetSummaries']))])==name_DataSet])==1

<a id='DataTemplate'></a>
+ ### Generate Template

Once we have generated the DataSet we must generate an analysis template, to do this we go to the QuickSight console and from there we generate a simple analysis, which we are going to call analysis_template_iris, with a dataset that has the same structure than our iris dataset so that an analysis could be created through SDK with our dataset created in this notebook.

Once our analysis is built by console, we generate the template using the function [create_template](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.create_template) To do this, first we need the ARN corresponding to the said analysis, which is obtained with the function [list_analyses](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.list_analyses) and the ARN corresponding to the dataset of the same analysis which is obtained from the function [list_data_sets](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.list_data_sets)


In [ ]:
response_list_analysis = qs_client.list_analyses(
    AwsAccountId=AwsAccountId
)

pos = np.where(np.array([response_list_analysis['AnalysisSummaryList'][i]['Name'] for i in np.arange(len(response_list_analysis['AnalysisSummaryList']))]) == analysis_template_name)[0][0]
ARN_analysis_template = response_list_analysis['AnalysisSummaryList'][pos]['Arn']

In [ ]:
response_list_datasets = qs_client.list_data_sets(
    AwsAccountId=AwsAccountId
)

pos = np.where(np.array([response_list_datasets['DataSetSummaries'][i]['Name'] for i in np.arange(len(response_list_datasets['DataSetSummaries']))]) == dataset_template_name)[0][0]
ARN_dataset_template = response_list_datasets['DataSetSummaries'][pos]['Arn']

In [ ]:
response_template = qs_client.create_template(
    AwsAccountId=AwsAccountId,
    TemplateId=TemplateId,
    Name=name_Template,
    SourceEntity= {
        'SourceAnalysis': {
            'Arn':ARN_analysis_template,
            'DataSetReferences': [
                {
                    'DataSetPlaceholder':dataset_template_name,
                    'DataSetArn':ARN_dataset_template
                },
            ]
        }
    },
    VersionDescription=Versiontemplate
)
response_template

<a id='DataAnalysis'></a>
+ ### Generate Analysis

After generating the template we use it to generate our analysis with the dataset that we generated in the notebook, the same one that we stored in the S3 bucket. At this point we use the function [create_analysis](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.create_analysis)


In [ ]:
Response_analysis = qs_client.create_analysis(
    AwsAccountId=AwsAccountId,
    AnalysisId=AnalysisId,
    Name=name_Analysis,
    Permissions=[
        {
            'Principal':arn_user,
            'Actions': [
                "quicksight:RestoreAnalysis",
                "quicksight:UpdateAnalysisPermissions",
                "quicksight:DeleteAnalysis",
                "quicksight:DescribeAnalysisPermissions",
                "quicksight:QueryAnalysis",
                "quicksight:DescribeAnalysis",
                "quicksight:UpdateAnalysis"
            ]
        }
    ],
    SourceEntity={
        'SourceTemplate':{
            'DataSetReferences':[
                {
                    'DataSetPlaceholder':dataset_template_name,
                    'DataSetArn':Response_dataset['Arn']
                },
            ],
            'Arn':response_template['Arn']
        }
    }
)
Response_analysis

<a id='DataDashboard'></a>
+ ### Generate Dashboard

After generating the template We use it to generate a dashboard with the dataset that we generated in the notebook, the same one that we stored in the S3 bucket. At this point we use the function [create_dashboard](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html#QuickSight.Client.create_analysis)



In [ ]:
Response_dashboard = qs_client.create_dashboard(
    AwsAccountId=AwsAccountId,
    DashboardId=DashboardId,
    Name=name_Dashboard,
    Permissions=[
        {
            'Principal':arn_user,
            'Actions':[
                'quicksight:DescribeDashboard',
                'quicksight:ListDashboardVersions',
                'quicksight:UpdateDashboardPermissions',
                'quicksight:QueryDashboard',
                'quicksight:UpdateDashboard',
                'quicksight:DeleteDashboard',
                'quicksight:DescribeDashboardPermissions',
                'quicksight:UpdateDashboardPublishedVersion'
            ]
        }
    ],
    SourceEntity={
        'SourceTemplate':{
            'DataSetReferences':[
                {
                    'DataSetPlaceholder':dataset_template_name,
                    'DataSetArn':Response_dataset['Arn']
                }
            ],
            'Arn':response_template['Arn']
        }
    },
    VersionDescription=VersionDashboard
)
Response_dashboard

<a id='Dropresources'></a>
+ ### Drop Resources

Finally, all the resources that were generated are eliminated.

In [ ]:
# drop datasource
qs_client.delete_data_source(
    AwsAccountId=AwsAccountId,
    DataSourceId=DataSourceId
)

In [ ]:
# drop dataset
qs_client.delete_data_set(
    AwsAccountId=AwsAccountId,
    DataSetId=DataSetId
)

In [ ]:
# drop template
qs_client.delete_template(
    AwsAccountId=AwsAccountId,
    TemplateId=TemplateId
)

In [ ]:
# drop analysis
qs_client.delete_analysis(
    AwsAccountId=AwsAccountId,
    AnalysisId=AnalysisId
)

In [ ]:
# drop Dashboard
qs_client.delete_dashboard(
    AwsAccountId=AwsAccountId,
    DashboardId=DashboardId
)

In [ ]:
# drop objects
s3 = boto3.resource('s3')
s3_bucket = s3.Bucket(bucket)

s3_bucket.delete_objects(
    Delete={
        'Objects':[
            {
                'Key':key_data_s3
            },
            {
                'Key':key_manifest_s3
            },
        ]
    }
)



In [ ]:
# drop bucket
s3_bucket.delete(
    ExpectedBucketOwner=AwsAccountId
)